Compose function - Line graph

In [ ]:
import torch
import numpy as np
from torch import nn

In [ ]:
class LineNetwork(nn.Module):
# Initialization
def init(self):
#Init
    super().__init__()
    #Variable - definition of sequence of neurons
    self.layers = nn.Sequential(
        #Receives entry and weight
        nn.Linear(1, 1)
)

#How the network computes
def forward(self, x):
return self.layers(x)

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch.distributions.uniform as urand

In [ ]:
class AlgebraicDataset(Dataset):
  #pass parameters: self, function, interval, sample
  def __init__(self, f, interval, nsamples):
    X = urand.Uniform(interval[0], interval[1]).sample([nsamples])
    self.data = [(x, f(x)) for x in X]

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    return self.data[idx]

In [ ]:
line = lambda x: 2*x + 3
interval = (-10, 10)
train_nsamples = 1000
test_nsamples = 100

In [ ]:
train_dataset = AlgebraicDataset(line, interval, train_nsamples)
test_dataset = AlgebraicDataset(line, interval, test_nsamples)

#Use batch to view little data
train_dataloader = DataLoader(train_dataset, batch_size=train_nsamples, shuffle=True)
test_dataloader = DataLoader(train_dataset, batch_size=train_nsamples, shuffle=True)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Run on {device}")
#See its run on GPU device

In [ ]:
model = LineNetwork().to(device)

In [ ]:
# loss function
# Mean Squared Error
lossfunc = nn.MSELoss()
# SGD = Stochastic Gradient Descent
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
# lr = learning rate

In [ ]:
def train(model, dataloader, lossfunc, optimizer):
  model.train()
  cumloss = 0.0
  for X, y in dataloader:
    X = X.unsqueeze(1).float().to(device)
    y = y.unsqueeze(1).float().to(device)

    pred = model(X)
    loss = lossfunc(pred, y)

    # clean cache
    optimizer.zero_grad()
    # compute gradients
    loss.backward()
    # reduces local error
    optimizer.step()

    # loss is a tensor; to get float
    cumloss += loss.item() 
  
  return cumloss / len(dataloader)


def test(model, dataloader, lossfunc):
  model.eval()
  
  cumloss = 0.0
  with torch.no_grad():
    for X, y in dataloader:
      X = X.unsqueeze(1).float().to(device)
      y = y.unsqueeze(1).float().to(device)

      pred = model(X)
      loss = lossfunc(pred, y)
      cumloss += loss.item() 
  
  return cumloss / len(dataloader)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# For view
def plot_comparinson(f, model, interval=(-10, 10), nsamples=10):
  fig, ax = plt.subplots(figsize=(10, 10))

  ax.grid(True, which='both')
  ax.spines['left'].set_position('zero')
  ax.spines['right'].set_color('none')
  ax.spines['bottom'].set_position('zero')
  ax.spines['top'].set_color('none')

  samples = np.linspace(interval[0], interval[1], nsamples)
  model.eval()
  with torch.no_grad():
    pred = model(torch.tensor(samples).unsqueeze(1).float().to(device))

  ax.plot(samples, list(map(f, samples)), "o", label="ground truth")
  ax.plot(samples, pred.cpu(), label="model")
  plt.legend()
  plt.show()

In [ ]:
#define amount and show
epochs = 201
for t in range(epochs):
train_loss = train(model, train_dataloader, lossfunc, optimizer)
if t % 10 == 0:
print(f"Epoch: {t}; Train Loss: {train_loss}")
plot_comparinson(line, model) 

test_loss = test(model, test_dataloader, lossfunc)
print(f"Test Loss: {test_loss}")

Expected result: 
First epoch step:

![Untitled](https://prod-files-secure.s3.us-west-2.amazonaws.com/3d7f32a3-eddd-46e3-a6ee-4a4f8ffd2aab/209c8c85-3354-4b40-86d8-037374f3af5b/Untitled.png)

Last epoch step:
![Untitled](https://prod-files-secure.s3.us-west-2.amazonaws.com/3d7f32a3-eddd-46e3-a6ee-4a4f8ffd2aab/e0ffb02b-9e76-4618-b768-6fa88a2aa6b9/Untitled.png)